In [21]:
# import libraries
from glob import glob
import os
import pandas as pd
import numpy as np

# Output table/file names
OH_SWITCHES_TABLE ='IN_OH_SW'
OH_TX_TABLE = 'IN_OH_TX'
UG_TX_TABLE = 'IN_UG_TX'
POLES_TABLE = 'IN_POLES'
UG_PRI_CABLE_TABLE = 'IN_CABLES'
NTWK_TX_TABLE = 'IN_NTWK_TX'

# asset_class_code(ACC) names
OH_SWITCHES_ASSET_CLASS ='OH_SWITCH'
OH_TX_ASSET_CLASS = 'OH_TX'
UG_TX_ASSET_CLASS = 'UG_TX'
POLES_ASSET_CLASS = 'POLE'
UG_PRI_CABLE_ASSET_CLASS = 'UG_CABLE'
NTWK_TX_ASSET_CLASS = 'NTWK_TX'


#Template folder
ASSET_TEMPLATE_FOLDER='AssetDataTemplates'

# #  First method to read col names: read from the entire folder
# # define filepath and sort the file list
# filesList = glob(os.path.join(ASSET_TEMPLATE_FOLDER, '*.xlsx'))
# numFiles = len(filesList)
# print('Number of Files: ', numFiles)
# sortedFileList = sorted(filesList)

# # read text files in tweet_input directory
# for f in sortedFileList:
#     fileName = os.path.basename(f).split('.')
#     print(fileName[0])
    
#     if ('$' not in fileName):
#         # Read CYME Feeder xlsx file into dataframes
#         with pd.ExcelFile(f) as xlsx:
#             dfTemp = pd.read_excel(xlsx, 'Sheet2') # 280 rows
            
#             # # Strip '\n' from column headers
#             dfTemp.rename(columns=lambda x: x.strip(), inplace=True)
    
#     #fileName[0]= dfTemp
#     #print(fileName[0].columns)
#     print(dfTemp.columns)
# Declare column names
UG_SWITCHES_COLS =['asset_id','id','asset_subclass_code','asset_class_code','install_year','hi',
                   'phasing','prid','circuit','tx_phase','in_valley','tie_feeder']

OH_TX_COLS = ['asset_id','asset_class_code','id','circuit','install_year','asset_subclass_code','hi',
              'phasing','primary_voltage','kva','tx_residential','tx_commercial','tx_industrial','device_residential',
              'device_commercial','device_industrial','upstream_device','prid','in_valley','pcb','banking']

UG_TX_COLS = ['asset_id','asset_subclass_code','asset_class_code','install_year','hi','phasing','prid','circuit',
              'primary_voltage','kva','in_valley','tx_residential','tx_commercial','tx_industrial','device_residential',
              'device_commercial','device_industrial','upstream_device','pcb','pedestal','switchable','switch_type','id']

UG_PRI_CABLE_COLS = ['asset_id','id','install_year','hi','asset_subclass_code','asset_class_code','phasing','prid',
                     'circuit','arrangement','installation','material','cable_size','config','length','num_splices',
                     'num_cables','prid_residential','prid_commercial','prid_industrial','nominal_voltage',
                     'wc_prid_catastrophic_res','wc_prid_catastrophic_comm','wc_prid_catastrophic_ind','cable_phase',
                     'wc_replacement','wc_switching_res','wc_switching_comm','wc_switching_ind','wc_switching_duration']

POLES_COLS = ['asset_id','asset_class_code','asset_subclass_code','install_year','hi character','phasing character',
              'prid character','pole_class','tx','tx_type','circuit1','circuit2','circuit3','circuit4','in_valley',
              'tx_residential','tx_commercial','tx_industrial','height','num_circuits','device','tx_kva','id','prid2',
              'prid3','prid4','tx_pcb']

NTWK_TX_COLS = ['asset_id','asset_subclass_code','asset_class_code','install_year','hi','phasing','prid','circuit',
                'primary_voltage','kva character','load double','id character','network_type','tx1','tx2','tx3','tx4']

# fileName - iterate through entire folder :)
fileName = 'Original_FiveAssetClasses.xlsx'
#fileNameOtherDevices = 'Other Device Numbers.xls'

# Read xlsx file into dataframes
with pd.ExcelFile(fileName) as xlsx:
    #dfTopology = pd.read_excel(xlsx, 'Topology', index_col=None, na_values=['NA']) # IGNORE for now
    dfTransformersV1 = pd.read_excel(xlsx, 'Transformers') # 280 rows
    dfSwitchesV1 = pd.read_excel(xlsx, 'Switches') # Tot:239 - R/Y/B: 116/108/103 values; based on phases
    dfPolesV1 = pd.read_excel(xlsx, 'Poles') # 239 rows; 'Spot Number\n' col contains unique tx ids
    dfCablesV1 = pd.read_excel(xlsx, 'UGPrimaryCables')
    dfFusesV1 = pd.read_excel(xlsx, 'Fuses') # 44 items
    dfUGStructuresV1 = pd.read_excel(xlsx,'UGStructures')


Summary = {'Transformers:': dfTransformersV1.shape, 'Switches:': dfSwitchesV1.shape,'Poles:': dfPolesV1.shape, 
           'Cables:': dfCablesV1.shape, 'Fuses:':dfFusesV1.shape, 'UGStructures:':dfUGStructuresV1.shape}
dfSummary = pd.DataFrame(Summary)

# Make one copy
dfTransformersV2 = dfTransformersV1
dfSwitchesV2 = dfSwitchesV1
dfPolesV2 = dfPolesV1
dfCablesV2 = dfCablesV1
dfFusesV2 = dfFusesV1
dfUGStructuresV2 = dfUGStructuresV1

# 17 columns dropped
dropCommonColumns = ['OBJECTID','WORKORDERID','FIELDVERIFY','COMMENTS','CREATIONUSER','DATECREATED','LASTUSER',
                     'DATEMODIFIED','WORKREQUESTID','DESIGNID','WORKLOCATIONID','WMSID','WORKFLOWSTATUS',
                     'WORKFUNCTION','GISONUMBER','GISOTYPENBR','OWNERSHIP']

def drop_columns(dfAssetClass, dropColumns):
    dfAssetClass = dfAssetClass.drop(dropColumns, axis=1)
    return dfAssetClass

#Drop all common columns 
dfSwitchesV2 = drop_columns(dfSwitchesV2, dropCommonColumns)
dfTransformersV2 = drop_columns(dfTransformersV2, dropCommonColumns)
dfFusesV2 = drop_columns(dfFusesV2,dropCommonColumns)
dfCablesV2 = drop_columns(dfCablesV2, dropCommonColumns)
dfUGStructuresV2 = drop_columns(dfUGStructuresV2, dropCommonColumns)
dfPolesV2 = drop_columns(dfPolesV2, dropCommonColumns)

# Make one copy
dfTransformers = dfTransformersV2
dfSwitches = dfSwitchesV2
dfPoles = dfPolesV2
dfCables = dfCablesV2
dfFuses = dfFusesV2
dfUGStructures = dfUGStructuresV2

SummaryV2 = {'Transformers:': dfTransformers.shape, 'Switches:': dfSwitches.shape,'Poles:': dfPoles.shape, 
           'Cables:': dfCables.shape, 'Fuses:':dfFuses.shape, 'UGStructures:':dfUGStructures.shape}
dfSummaryV2 = pd.DataFrame(SummaryV2)
print(dfSummary)
print(dfSummaryV2)
# print(dfSummary-dfSummaryV2) # shows 17 cols dropped      

   Cables:  Fuses:  Poles:  Switches:  Transformers:  UGStructures:
0     3865     736   18961        537           3618          16883
1       39      46      31         52             57             40
   Cables:  Fuses:  Poles:  Switches:  Transformers:  UGStructures:
0     3865     736   18961        537           3618          16883
1       22      29      14         35             40             23


In [22]:
# Save future wait times while running
dfTransformers = dfTransformersV2
dfSwitches = dfSwitchesV2
dfPoles = dfPolesV2
dfCables = dfCablesV2
dfFuses = dfFusesV2
dfUGStructures = dfUGStructuresV2

# Switches
fileNameOtherDevices = 'Other Device Numbers.xlsx'
# Read Other Device Numbers into dataframes
with pd.ExcelFile(fileNameOtherDevices) as xls:
    dfSwitchGears = pd.read_excel(xls, 'SWITCHGEARS') # 280 rows

dropSGcols = ['Switch Gear', 'Adrs #','Location','City','Notes','To Type','Inst. Date','Mftr.','Catalog#','Serial#',
             'DOM','Comments']

dfSwitchGears = drop_columns(dfSwitchGears, dropSGcols)
#dfSwitchGears = dfSwitchGears.dropna() # drop all rows with NaN values

def new_columns(dfAssetClass, numAssetRows, columnID):
    dfAssetClass[columnID] = pd.DataFrame(np.empty([numAssetRows,1]).cumsum(axis=1))
    dfAssetClass.loc[:,columnID] = np.nan
    return dfAssetClass[columnID]

dictSGassetSubclass = {'PMH-3':'AIR_INSULATED_LIVEFRONT','PMH-5':'AIR_INSULATED_LIVEFRONT',
                       'PMH-9':'AIR_INSULATED_LIVEFRONT','PMH-11':'AIR_INSULATED_LIVEFRONT',
                       'PME-9':'AIR_INSULATED_DEADFRONT','PME-10':'AIR_INSULATED_DEADFRONT',
                       'PME-11':'AIR_INSULATED_DEADFRONT','VISTA-321':'SF6_INSULATED_SWITCH',
                       'VISTA-422':'SF6_INSULATED_SWITCH','VISTA-431':'SF6_INSULATED_SWITCH',
                       '422':'SC_ELEC','431':'SC_ELEC','321':'SC_ELEC','G&W':'GW',
                       'NET':'CARTE_ELEC_LTD'}
# 'Type' -> 'PMH'
# 'Loc_No' -> '149-S'
dfSwitchGears['Type'] = dfSwitchGears['Type'].fillna(method='ffill')
numSGrows = len(dfSwitchGears['Loc_No'])
dfSwitchGears['Subclass'] = new_columns(dfSwitchGears, numSGrows, 'Subclass')
dfSwitchGears =dfSwitchGears.astype(str)
#dfSwitchGears['Loc_No'] = dfSwitchGears.iloc[:,'Loc_No'].apply[s.lstrip("0") for s in listOfNum]
dfSwitchGears['Loc_No'] = [s.lstrip("0") for s in dfSwitchGears['Loc_No']]
dfSwitchGears['Subclass'] = dfSwitchGears['Type'].apply(lambda x: dictSGassetSubclass[x])

###############
# 2
###############

dropSwitchesCols = ['ANCILLARYROLE','ENABLED','FEEDERINFO','ELECTRICTRACEWEIGHT','LOCATIONID','GPSDATE','LABELTEXT',
                    'OPERATINGVOLTAGE', 'NOMINALVOLTAGE', 'MAXOPERATINGVOLTAGE','MAXCONTINUOUSCURRENT','PRESENTPOSITION_R', 
                    'PRESENTPOSITION_Y', 'PRESENTPOSITION_B','NORMALPOSITION_R','NORMALPOSITION_Y','NORMALPOSITION_B', 
                    'SCADACONTROLID', 'SCADAMONITORID','PREFERREDCIRCUITSOURCE','TIESWITCHINDICATOR',
                    'GANGOPERATED', 'MANUALLYOPERATED','FEATURE_STATUS','HYPERLINK','HYPERLINK_PGDB','SYMBOLROTATION',
                    'INSULATOR_MATERIAL']

# drop columns and drop rows that are not UG Switch
dfSwitches = drop_columns(dfSwitches,dropSwitchesCols)
dfSwitches = dfSwitches[dfSwitches.SUBTYPECD == 6]
# df.query('line_race != 0')
# df = df[df.line_race != 0]

# Rename Switch columns
dfSwitches = dfSwitches.rename(columns={'SUBTYPECD':'Asset_Class_Code',
                                        'DEVICENUMBER':'ID',
                                        'COMPATIBLEUNITID':'Asset_Subclass_Code',
                                        'PHASEDESIGNATION':'PHASING',
                                        'FEEDERID':'CIRCUIT', 
                                        'FEEDERID2':'TIE_FEEDER',
                                        'INSTALLATIONDATE':'INSTALL_YEAR'})
# Separate year
dfSwitches['INSTALL_YEAR'] = dfSwitches['INSTALL_YEAR'].apply(lambda x: x.year)

# Add additional columns and fill with NaNs
numSwitchRows = len(dfSwitches['ID'])
dfSwitches['HI'] = new_columns(dfSwitches,numSwitchRows, 'HI')
dfSwitches['TX_PHASE'] = new_columns(dfSwitches,numSwitchRows, 'TX_PHASE')
dfSwitches['IN_VALLEY'] = new_columns(dfSwitches,numSwitchRows, 'IN_VALLEY')
dfSwitches['PRID'] = new_columns(dfSwitches,numSwitchRows, 'PRID')

print(dfSwitches.shape) # (537, 11) with all switches, (260,11) with subtype=6 ('Switch Switchgear')
print('dfSG Shape:', dfSwitchGears.shape) #(111, 3)

###############
# 3
###############

# Replace 'Asset Subclass' col with actual names
dfSwitches['ID'] = dfSwitches['ID'].astype(str)
dfSwitches=pd.merge(dfSwitches, dfSwitchGears, how='left', left_on='ID', right_on='Loc_No')
#dfSwitches['ID'] = dfSwitchGears['Loc_No'].apply(lambda x: )
#df.merge(df1, on='sku', how='left')
# print(len(pd.unique(dfSwitchGears['Loc_No'].values.ravel()))) # 111

switchesDropMoreCols = ['Asset_Subclass_Code', 'Loc_No']
dfSwitches = dfSwitches.drop(switchesDropMoreCols, axis=1)
dfSwitches = dfSwitches.rename(columns={'Subclass':'Asset_Subclass_code'})

# Lower case column names
dfSwitches.columns = map(str.lower, dfSwitches.columns)

# Rename proper asset nomenclature
dfSwitches['asset_class_code'] = OH_SWITCHES_ASSET_CLASS


# Phasing change - need to change it to 'str' type, int/float dict key lookup doesn't work
# [Ian: 1,2 and 4 are single phase, 3,5 and 6 are two phase and 7 is three phase]
phasingDict = {'1.0': '1Ph', '2.0':'1Ph','4.0':'1Ph','3.0':'2Ph','5.0':'2Ph','6.0':'2Ph','7.0':'3Ph'}
#phasingDict = {'1': '1Ph', '2':'1Ph','4.0':'1Ph','3':'2Ph','5':'2Ph','6':'2Ph','7':'3Ph'}
dfSwitches['phasing'] = dfSwitches['phasing'].astype(str)
dfSwitches['phasing'] = dfSwitches['phasing'].apply(lambda x: phasingDict[x])
#print(dfSwitches.dtypes)

#Rearrange columns
# dfSwitches=dfSwitches[['Asset_Class', 'Asset_Subclass', 'ID','CIRCUIT','INSTALL_YEAR','TIE_FEEDER','PHASING',
#                        'IN_VALLEY','HI','TX_PHASE','PRID','Type']]
# UG_SWITCHES_COLS =['asset_id','id','asset_subclass_code','asset_class_code','install_year','hi',
#                    'phasing','prid','circuit','tx_phase','in_valley','tie_feeder']
dfSwitches =dfSwitches[['id','asset_subclass_code','asset_class_code','install_year','hi','phasing','prid','circuit','tx_phase','in_valley','tie_feeder','type']]



# Write to spreadsheet
MasterFile = pd.ExcelWriter('V3_GIS_Switches.xlsx')
dfSwitches.to_excel(MasterFile, 'Sheet1')
MasterFile.save()


(260, 11)
dfSG Shape: (451, 3)
